In [ ]:
pip install timm

In [ ]:
import os
import numpy as np
import cv2
import csv
from glob import glob
import torch
import matplotlib.pyplot as plt
import gc
import sys
import cv2
import urllib.request

model_type = "DPT_Large" 
depth_estimator = torch.hub.load("intel-isl/MiDaS", model_type)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
depth_estimator.to(device)
depth_estimator.eval()
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.dpt_transform

In [ ]:
path = "../input/image-matching-challenge-2022/train/"
lst = os.listdir(path)
if not os.path.exists("depth_maps"):
    os.mkdir("depth_maps")
for _folder in lst:
    if os.path.isdir(path + _folder):
        if not os.path.exists("depth_maps/" + _folder):
            os.mkdir("depth_maps/" + _folder)
        lst2 = os.listdir(path + _folder + "/images/")
        for fname in lst2:
            img = cv2.imread(path + _folder + "/images/" + fname)
            scale = 640 / max(img.shape[0], img.shape[1]) 
            w = int(img.shape[1] * scale)
            h = int(img.shape[0] * scale)
            img = cv2.resize(img, (w, h))
            input_batch = transform(img).to(device)
            with torch.no_grad():
                mask = depth_estimator(input_batch)

                mask = torch.nn.functional.interpolate(
                    mask.unsqueeze(1),
                    size=img.shape[:2],
                    mode="bicubic",
                    align_corners=False,
                ).squeeze()

            mask = mask.cpu().numpy()
            cv2.imwrite("depth_maps/" + _folder + "/" + fname, mask)
            break